# 학습 알고리즘 구현하기
- 미니배치
- 기울기 산출
- 매개변수 갱신
- 반복

확률적 경사하강법(SGD : Stochastic Gradient Descent)의 구현 : 확률적이라고 함은 미니배치를 의미하는데 확률적으로 무작위로 골라낸 데이터에 대해서 수행하는 경사하강법이라는 의미이다.

넘파이의 축(axis) 개념 이해하기 : https://pybasall.tistory.com/129

In [2]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient

### 2층 신경망 클래스 구현하기

In [3]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params["b1"] = weight_init_std * np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params["b2"] = weight_init_std * np.zeros(output_size)
        
    def predict(self,x):
        W1, W2 = self.params["W1"],self.params["W2"]
        b1, b2 = self.params["b1"],self.params["b2"]
        
        a1 = np.dot(x,W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self,x,t):
        y = self.predict(x)
        return cross_entropy_error(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.armgax(t,axis=1)
        # y와 t의 레이블이 일치하면 정답인 것
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self,x,t):
        def loss_W(W):
            return self.loss(x,t)
        
        grads = {}
        grads["W1"] = numerical_gradient(loss_W,self.params["W1"])
        grads["b1"] = numerical_gradient(loss_W,self.params["b1"])
        grads["W2"] = numerical_gradient(loss_W,self.params["W2"])
        grads["b2"] = numerical_gradient(loss_W,self.params["b2"])
        
        return grads

In [4]:
net = TwoLayerNet(input_size=784,hidden_size=100,output_size=10)
print(net.params["W1"].shape)
print(net.params["b1"].shape)
print(net.params["W2"].shape)
print(net.params["b2"].shape)
# params 딕셔너리에 저장된 매개변수를 토대로 예측이 처리된다

(784, 100)
(100,)
(100, 10)
(10,)


In [5]:
x = np.random.rand(100,784)
y = net.predict(x) # 100장, 각 784개의 데이터 원소가 담긴 x에 대해서 예측을 진행할 수 있다.
print(y.shape)

(100, 10)


In [11]:
x = np.random.rand(100,784)
t = np.random.rand(100,10)

grads = net.numerical_gradient(x,t)
print(grads["W1"].shape)
print(grads["b1"].shape)
print(grads["W2"].shape)
print(grads["b2"].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [19]:
grads["W1"][0][0]
# 입력층과 은닉층 사이의 매개변수 w11은 0.0001 이기에 손실함수 값을 증가시키기에 음의 방향으로 갱신시켜야 감소시킬 것

0.00010674752415340549

### 미니배치 학습 구현하기
- 훈련 데이터 중 일부 데이터를 무작위로 꺼내는 '미니배치'
- 미니배치에 대해서 경사하강법으로 손실함수를 줄이는 방향으로 매개변수 개선해보기

In [ ]:
import numpy as np
from dataset.mnist import load_mnist

(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_lst = []
# 하이퍼파라미터 : 사람이 설정해야 하는 파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1 # 학습률

network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

for i in range(iters_num): # iter_nums 만큼 미니배치 학습을 진행
    batch_idx = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_idx] ; t_batch = t_train[batch_idx]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch,t_batch)
    
    # W1 W2 b1 b2에 대해서 각각 경사하강법 적용
    for key in ("W1","b1","W2","b2"):
        network.params[key] -= learning_rate * grad[key]
        # 매개변수 딕셔너리인 params를 갱신해줘야 한다.
        # 그래야 loss 값이 갱신된다.
        
    loss = network.loss(x_batch,t_batch)
    train_loss_lst.append(loss)

epoch(에포크)마다 시험데이터로 평가하기
- 위 코드와 거의 동일한데 시험데이터로 평가하는 것만 추가
- epoch란? 1epoch는 학습에서 훈련데이터를 모두 소진한 횟수.
- 1 batch_size 100으로 6만개의 데이터를 모두 소진하려면 600번 학습해야함
- 이 경우에는 600번 학습하면 그것이 1epoch인 것

In [ ]:
import numpy as np
from dataset.mnist import load_mnist

(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_lst = []
train_acc_lst = [] # 학습정확도
test_acc_lst = [] # 시험정확도

# 하이퍼파라미터 : 사람이 설정해야 하는 파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1 # 학습률

network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

for i in range(iters_num): # iter_nums 만큼 미니배치 학습을 진행
    batch_idx = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_idx] ; t_batch = t_train[batch_idx]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch,t_batch)
    
    # W1 W2 b1 b2에 대해서 각각 경사하강법 적용
    for key in ("W1","b1","W2","b2"):
        network.params[key] -= learning_rate * grad[key]
        # 매개변수 딕셔너리인 params를 갱신해줘야 한다.
        # 그래야 loss 값이 갱신된다.
        
    loss = network.loss(x_batch,t_batch)
    train_loss_lst.append(loss)
    
    iter_per_epoch = max(train_size/batch_size,1) # 6만 / 100 -> 1epoch 즉, 학습데이터를 모두 소진시키려면 600번 반복
    
    # 1 epoch 당 정확도 계산 -> 1epoch 600번
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_batch,t_batch)
        test_acc = network.accuracy(x_test,t_test) # 갱신된 매개변수에 시험데이터 적용
        train_acc_lst.append(train_acc)
        test_acc_lst.append(test_acc)

In [2]:
import numpy as np

a = np.array([[[1,2],[3,4]],[[3,4],[6,7]]])

In [7]:
np.max(a,axis=0)

array([[3, 4],
       [6, 7]])

In [15]:
import numpy as np
a3 = np.array([[[7,1],[9,4],[2,3]],
               [[4,1],[0,4],[8,0]],
               [[1,1],[3,4],[6,9]],
               [[5,1],[8,4],[4,7]]])
np.sum(a3,axis=0)

array([[17,  4],
       [20, 16],
       [20, 19]])

In [16]:
np.sum(a3,axis=1)

array([[18,  8],
       [12,  5],
       [10, 14],
       [17, 12]])

In [17]:
np.sum(a3,axis=2)

array([[ 8, 13,  5],
       [ 5,  4,  8],
       [ 2,  7, 15],
       [ 6, 12, 11]])